# SD Neighborhood Pulse Phase 3 Notebook\nQuick checks for frustration index and hotspot outputs built in dbt.

In [ ]:
from pathlib import Path\nimport duckdb\nimport pandas as pd\nimport plotly.express as px\n\nDB_PATH = Path('../data/db/city_health.duckdb')\nassert DB_PATH.exists(), f'Missing DuckDB file: {DB_PATH.resolve()}'

In [ ]:
con = duckdb.connect(str(DB_PATH), read_only=True)\nfrustration = con.execute('select * from fct_neighborhood_frustration_index').fetchdf()\nhotspots = con.execute('select * from fct_request_hotspots').fetchdf()\nfrustration.head()

In [ ]:
latest = frustration['as_of_date'].max()\nfocus = frustration[(frustration['as_of_date'] == latest) & (frustration['window_days'] == 30) & (frustration['grain_type'] == 'comm_plan_name')]\nfocus = focus.sort_values('frustration_index', ascending=False).head(20)\nfig = px.bar(focus, x='frustration_index', y='grain_value', orientation='h', title='Top 20 comm_plan_name (30d)')\nfig.update_layout(yaxis={'categoryorder': 'total ascending'})\nfig

In [ ]:
hotspots.sort_values(['window_days', 'request_count'], ascending=[True, False]).head(25)